In [1]:
"""
This little program is a crawler which crawls the data from etw.nextdigital.com.hk and extract all the 
restaurants information
- beautiful soup was used to crawl infinite scrolling page
- regex is used to extract values
- data are stored in both local JSON fire and in SQLite DB
"""

'\nThis little program is a crawler which crawls the data from etw.nextdigital.com.hk and extract all the \nrestaurants information\n- beautiful soup was used to crawl infinite scrolling page\n- regex is used to extract values\n- data are stored in both local JSON fire and in SQLite DB\n'

In [2]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import json
import ssl
import re
import time
import sqlite3
import os


In [3]:
def loadMasterJSON(filePath, jsonFileName):
    with open(filePath + jsonFileName) as fp:
        f = json.load(fp)
    return f

In [4]:
def createTableAndWriteData(db, data_elements, data_cat):
    # define DB path and name
    c = db.cursor()
    
    table_name = data_cat.upper()
    key_field = data_cat.lower() + '_id'
    value = data_cat.lower()
    
    tableSQL = ("CREATE TABLE IF NOT EXISTS " + table_name 
                + " (" + key_field + " INTEGER NOT NULL UNIQUE PRIMARY KEY AUTOINCREMENT," 
                + value + " varchar(128) NOT NULL UNIQUE);")
    
    # try to create 
    try:
        c.execute(tableSQL)
    except ValueError:
        print(ValueError)
    
    
    count = 0

    for data in data_elements:
        query = "INSERT OR REPLACE into " + table_name + " values (null,'" + data + "')"
        try:
            c.execute(query)
        except ValueError:
            print(ValueError)
            continue
        count += 1
    
    print(str(count) + " records added to table " + table_name)
    
    db.commit()
    c.close()
        
    return

In [5]:
def retrieveDB(db, data_cat):
    c = db.cursor()
    elements = []
    query = "SELECT " + data_cat + " from " + data_cat.upper() 
    
    try:
        elements = c.execute(query).fetchall()
        
        #db.row_factory = lambda cursor, row: row[0]
        elements = c.execute(query).fetchall()
        elementList = [ls[0] for ls in elements]
        
    except ValueError:
        print(ValueError)
    
    return elementList

In [6]:
def prepareMasterSeedList(district_elements, cuisine_elements, foodtype_elements, updateDB = True):
    MasterSeedURL = []
    baseURL = "http://etw.nextdigital.com.hk/search/restaurant/?"
    
        
    
    for i in range(len(district_elements)):
        d = baseURL + "district=" + district_elements[i]
        MasterSeedURL.append(d)

    for i in range(len(cuisine_elements)):
        d = baseURL + "cuisine=" + cuisine_elements[i]
        MasterSeedURL.append(d)

    for i in range(len(foodtype_elements)):
        d = baseURL + "foodtype=" + foodtype_elements[i]
        MasterSeedURL.append(d)   

    print("Masterlist length: " +  str(len(MasterSeedURL)))
        
        
    return MasterSeedURL

In [7]:
def crawlMetaInfo(url):
    
    browser = webdriver.Chrome('/Users/samuelpun_old/Desktop/MLfolders/chromedriver')

    browser.get(url)
    time.sleep(1)

    elem = browser.find_element_by_tag_name("body")

        
    # extract Search Options seeds
    district_elements = extractElements(browser, "selected_district", "value", "find_elements_by_name")
    cuisine_elements = extractElements(browser, "selected_cuisine", "value", "find_elements_by_name")
    foodtype_elements = extractElements(browser, "selected_foodtype", "value", "find_elements_by_name")
    
    browser.close()
                                             
    return district_elements, cuisine_elements, foodtype_elements

In [8]:
def extractElements(browserObj, tagName, attr, method):
    
    if method == "find_elements_by_name":
        elements = browserObj.find_elements_by_name(tagName)
    elif method  == "find_elements_by_class_name":
        elements = browserObj.find_elements_by_class_name(tagName)
        
    snippets = []
    
    for item in elements:
        snippets.append(item.get_attribute(attr))
    
    return snippets

In [9]:
def crawlRestInfo(url, tag, pageDownNumber = 100, pageDownSleepTime = 5.0, infScroll = False):
    
    if infScroll == False:
        ctx = ssl.create_default_context()
        ctx.check_hostname = False
        ctx.verify_mode = ssl.CERT_NONE
    
        html = urllib.request.urlopen(url, context=ctx).read()
        soup = BeautifulSoup(html, 'html.parser')
    
        # Retrieve all of the anchor tags
        snippets = soup(tag)
    
    else:
        browser = webdriver.Chrome('/Users/samuelpun_old/Desktop/MLfolders/chromedriver')

        browser.get(url)
        time.sleep(pageDownSleepTime)

        elem = browser.find_element_by_tag_name("body")

        # Generate auto pagedown
        lastHeight = browser.execute_script("return document.body.scrollHeight")
        while pageDownNumber:
            
            elem.send_keys(Keys.PAGE_DOWN)
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5.0)
            newHeight = browser.execute_script("return document.body.scrollHeight")
            if newHeight == lastHeight:
                break
            else:
                lastHeight = newHeight
            pageDownNumber-=1

        # extract restaurants class
        restHTML_elementsList = extractElements(browser, "result-restaurant-list", 
                                             "innerHTML", "find_elements_by_class_name")
        browser.close()
        
                                             
    return restHTML_elementsList

In [10]:
def addrToDistrictID(addr, district_elements):
    
    db = sqlite3.connect(filePath + dbName)
    c = db.cursor()
    district_id = ""
    
    l = len(addr)
    for i in range(l):
        substr = addr[0:1-i]
        if substr in district_elements:
            query = "SELECT DISTRICT.district_id from DISTRICT where DISTRICT.district = '" + substr + "'"
            district_id = c.execute(query).fetchall()
            district_id = [ls[0] for ls in district_id]
            #print(district_id)
    
    if len(district_id) > 0:
        db.close()
        district_id = district_id[0]
    else:
        print(addr)
        district_id = 1
    
    return district_id

In [11]:
def districtToDistrict_ID(district, district_elements):
    
    db = sqlite3.connect(filePath + dbName)
    c = db.cursor()
    district_id = ""
    
    if district in district_elements:
        query = "SELECT DISTRICT.district_id from DISTRICT where DISTRICT.district = '" + district + "'"
        district_id = c.execute(query).fetchall()
        district_id = [ls[0] for ls in district_id]
            #print(district_id)
    
    if len(district_id) > 0:
        db.close()
        district_id = district_id[0]
    else:
        print(addr)
        district_id = 1
    
    return district_id

In [12]:
def RestHTMLtoList(restHTML_elementsList, restMasterList, district_elements):
    
    count = 0
    
    # from a list of href exact restaurant and new masterList info
    for i in range(len(restHTML_elementsList)):
        #print(restHTML_elementsList[0])
        restID = re.findall('\\/restaurant\\/([0-9]+)\\/.*\\"',str(restHTML_elementsList[i]))
        url_tags = re.findall('\\/restaurant\\/[0-9]+\\/(.*)\\"',str(restHTML_elementsList[i]))

        for u in url_tags[0].split('-'):
            if u in district_elements:
                district_id = districtToDistrict_ID(u, district_elements)
        
        name = re.findall('ar-name\\"\\>(.+)\\<\\/div\\>',str(restHTML_elementsList[i]))
        tel = re.findall('ar-tel\\"\\>([0-9]+\s[0-9]+)\\<\\/div\\>',str(restHTML_elementsList[i]))
        addr = re.findall('ar-address\\"\\>(.+)\\<\\/div\\>',str(restHTML_elementsList[i]))
        tags = re.findall('ar-cuisine\\"\\>\\\n\s+(.+)\\<\\/div\\>',str(restHTML_elementsList[i]))
        rating = re.findall('editorialrating\\"\\>(\S+)\\<\\/span\\>',str(restHTML_elementsList[i]))


        rest = {}
        
        if len(restID) > 0:
            #print(rest)
            rest['ID'] = restID[0]
            
        if len(tags) > 0:
            tagsList = tags[0].split('|')
            for i in range(len(tagsList)):
                tagsList[i] = tagsList[i].strip()
            rest['tags'] = tagsList
            #print(rest['tags'])
            
        if len(name) > 0: 
            rest['name'] = name[0]
            
        
        if len(rating) > 0:     
            rest['rating'] = rating[0]
        else:
            rest['rating'] = ''
        
        
        if len(tel) > 0:     
            rest['tel'] = tel[0]
        else:
            rest['tel'] = ''
        
        if len(addr) > 0: 
            rest['addr'] = addr[0]
            
            """
            #remove '香港' if it is in the address
            if '香港' in addr[0] and not ('香港仔' in addr[0]):
                start = addr[0].index('香港') + len('香港')
                newadd = addr[0][start: -1]
            else:
                newadd = addr[0]

            district_id = addrToDistrictID(newadd, district_elements)
            """
        else:
            rest['addr'] = ''
        
        if len(str(district_id)) > 0: 
            rest['district_id'] = district_id
        else:
            rest['district_id'] = ''
            print("no district_id found")
            
        if restID[0] in restMasterList:
            continue
        else:
            #print(restID[0])
            #print(rest)
            restMasterList[restID[0]] = rest
            count = count + 1
    
    print(str(len(restHTML_elementsList)) + " records found. " + str(count) + 
          " records updated. Total record: " + str(len(restMasterList)))

         
    return restMasterList

In [13]:
def processCrawlList(filePath, seedMasterList, crawlAndSaveOnly, district_elements, restMasterList = {}):
    
    pageDownNumber = 2000
    pageDownSleepTime = 5.0
    saveLocalFile = True
    
    # scan thru the seedMasterList go get a list of href
    url_count = 0
    
    for i in range(len(seedMasterList)):
        print("fetching " + str(i) + "th URL out of " + str(len(seedMasterList)))
        this_url = seedMasterList[i]
        restHTML_elementsList = crawlRestInfo(this_url, 'a', pageDownNumber, pageDownSleepTime, True) 
        
        if saveLocalFile == True:
            fileCount = 0
            for rest_snippet in range(len(restHTML_elementsList)):
                with open(filePath + str(i) + "-" + str(rest_snippet) + ".txt", "w") as file:
                    file.write(restHTML_elementsList[rest_snippet])
                    fileCount += 1
            print(str(fileCount) + " files saved to: " + str(filePath))
        
        if crawlAndSaveOnly == False:
            restMasterList = RestHTMLtoList(restHTML_elementsList, restMasterList, district_elements)
        
        url_count += 1
    
    print(str(url_count) + " url fetched")

    return restMasterList

In [14]:
def loadTxtFiletoElementsList(filePath):
    
    restHTML_elementsList = []
    
    for file in os.listdir(filePath):
        if file.endswith(".txt"):
            #print(os.path.join(filePath, file))
            with open(filePath + file, "r") as file:
                rest_snippet = file.read()
                restHTML_elementsList.append(rest_snippet)
    
    print(str(len(restHTML_elementsList)) + " files appended.")
    
    return restHTML_elementsList

In [15]:
def saveData(filePath, dbName, jsonFileName, restMasterList, JSON = True, DB = True):
    
    if JSON == True:
        # save file as JSON format
        with open((filePath + jsonFileName), 'w+') as fp:
            json.dump(restMasterList, fp)
            print( str(len(restMasterList)) + " records saved to local JSON file: " + jsonFileName)
            #existingRecord = dict(json.load(fp))
    
    if DB == True:
        db = sqlite3.connect(filePath + dbName)
        c = db.cursor()
        
        create_table_sql = """CREATE TABLE IF NOT EXISTS ETWRestsDB (
                                        rest_id text PRIMARY KEY NOT NULL,
                                        addr text,
                                        name text NOT NULL,
                                        rating text,
                                        tags text,
                                        tel text,
                                        district_id INTEGER,
                                        foreign key(district_id) REFERENCES DISTRICT(district_id)
                                    );"""
        try:
            c.execute('DROP TABLE IF EXISTS ETWRestsDB;')
            c.execute(create_table_sql)
        except ValueError:
            print(ValueError)
        
        
        count = 0
        
        query = "INSERT or REPLACE into ETWRestsDB values (?,?,?,?,?,?,?)"
        columns = ['addr', 'name', 'rating', 'tags', 'tel', 'district_id']
        for restid, data in restMasterList.items():
            keys = (restid,) + tuple(str(data[c]) for c in columns)
            #print(str(keys))
            c.execute(query, keys)
            count = count + 1
        
        db.commit()
        c.close()
        print(str(count) + " number of records inserted in DB")
    
    return
    

In [ ]:
def updateRelTable(filePath, dbName, restMasterList):
    
    db = sqlite3.connect(filePath + dbName)    
    c = db.cursor()
    
    tableSQL_rest_cuisine = """CREATE TABLE IF NOT EXISTS REST_CUISINE ( 
                                    rest_id TEXT NOT NULL ,
                                    cuisine_id INTEGER NOT NULL,
                                    PRIMARY KEY(rest_id, cuisine_id),
                                    foreign key(rest_id) REFERENCES ETWRestsDB(rest_id),
                                    foreign key(cuisine_id) REFERENCES CUISINE(cuisine_id)
                                    );"""

    tableSQL_rest_foodtype = """CREATE TABLE IF NOT EXISTS REST_FOODTYPE ( 
                                    rest_id TEXT NOT NULL ,
                                    foodtype_id INTEGER NOT NULL,
                                    PRIMARY KEY(rest_id, foodtype_id),
                                    foreign key(rest_id) REFERENCES ETWRestsDB(rest_id),
                                    foreign key(foodtype_id) REFERENCES FOODTYPE(foodtype_id)
                                    );"""    
       
    c.execute(tableSQL_rest_cuisine)
    c.execute(tableSQL_rest_foodtype)
    

    for restid, data in restMasterList.items():
        #print(str(restid) + ": ")
        for t in data['tags']:
            f_query = "select foodtype_id from FOODTYPE where foodtype = '" + t + "'"
            f_id = c.execute(f_query).fetchall()
            foodtype_id = [ls[0] for ls in f_id]
            if len(foodtype_id) > 0:
                query = ("INSERT OR REPLACE into REST_FOODTYPE values (" + str(restid) 
                         + "," + str(foodtype_id[0]) + ");")
                try:
                    c.execute(query)
                except ValueError:
                    print(ValueError)
            
            c_query = "select cuisine_id from CUISINE where cuisine = '" + t + "'"
            c_id = c.execute(c_query).fetchall()
            cuisine_id = [ls[0] for ls in c_id]
            if len(cuisine_id) > 0:
                query = ("INSERT OR REPLACE into REST_CUISINE values (" 
                         + str(restid) + "," + str(cuisine_id[0]) + ");")    
                try:
                    c.execute(query)
                except ValueError:
                    print(ValueError)
    db.commit()
    c.close()               

    return

In [ ]:
#url = input('Enter - ')
crawlAndUpdateMetaInfo = True
crawlAndSaveOnly = False
filePath = '/Users/samuelpun_old/Desktop/MLfolders/ETWdata/'
dbName = "ETW1.db"
jsonFileName = "restMasterList.json"

seedMasterList = []

# Extract Search Meta Data
if crawlAndUpdateMetaInfo == True:
    #crawl the data elements
    district_elements, cuisine_elements, foodtype_elements = crawlMetaInfo("http://etw.nextdigital.com.hk")
    #write to DB
    db = sqlite3.connect(filePath + dbName)
    createTableAndWriteData(db, district_elements, 'district')
    createTableAndWriteData(db, cuisine_elements, 'cuisine')
    createTableAndWriteData(db, foodtype_elements, 'foodtype')
    db.close()
else:
    # retrieve from db
    db = sqlite3.connect(filePath + dbName)
    district_elements = retrieveDB(db, 'district')
    cuisine_elements = retrieveDB(db, 'cuisine')
    foodtype_elements = retrieveDB(db, 'foodtype')
    db.close()

# Prepare Master Crawl List
seedMasterList = prepareMasterSeedList(district_elements, cuisine_elements, foodtype_elements)

# Crawl the MasterList  seedMasterList[10:12]
restMasterList = processCrawlList(filePath, seedMasterList, crawlAndSaveOnly, district_elements)

# Load from DB
if crawlAndSaveOnly == True:
    # dont expect to get direct list from the previous process
    restHTML_elementsList = loadTxtFiletoElementsList(filePath)
    restMasterList = RestHTMLtoList(restHTML_elementsList, restMasterList, district_elements)

saveData(filePath, dbName, jsonFileName, restMasterList)
updateRelTable(filePath, dbName, restMasterList)

91 records added to table DISTRICT
68 records added to table CUISINE
94 records added to table FOODTYPE
Masterlist length: 253
fetching 0th URL out of 253
0 files saved to: /Users/samuelpun_old/Desktop/MLfolders/ETWdata/
0 records found. 0 records updated. Total record: 0
fetching 1th URL out of 253
515 files saved to: /Users/samuelpun_old/Desktop/MLfolders/ETWdata/
515 records found. 515 records updated. Total record: 515
fetching 2th URL out of 253
862 files saved to: /Users/samuelpun_old/Desktop/MLfolders/ETWdata/
862 records found. 862 records updated. Total record: 1377
fetching 3th URL out of 253
401 files saved to: /Users/samuelpun_old/Desktop/MLfolders/ETWdata/
401 records found. 401 records updated. Total record: 1778
fetching 4th URL out of 253
164 files saved to: /Users/samuelpun_old/Desktop/MLfolders/ETWdata/
164 records found. 164 records updated. Total record: 1942
fetching 5th URL out of 253
72 files saved to: /Users/samuelpun_old/Desktop/MLfolders/ETWdata/
72 records fo

142 files saved to: /Users/samuelpun_old/Desktop/MLfolders/ETWdata/
142 records found. 142 records updated. Total record: 6240
fetching 54th URL out of 253
12 files saved to: /Users/samuelpun_old/Desktop/MLfolders/ETWdata/
12 records found. 12 records updated. Total record: 6252
fetching 55th URL out of 253
35 files saved to: /Users/samuelpun_old/Desktop/MLfolders/ETWdata/
35 records found. 35 records updated. Total record: 6287
fetching 56th URL out of 253
22 files saved to: /Users/samuelpun_old/Desktop/MLfolders/ETWdata/
22 records found. 22 records updated. Total record: 6309
fetching 57th URL out of 253
82 files saved to: /Users/samuelpun_old/Desktop/MLfolders/ETWdata/
82 records found. 82 records updated. Total record: 6391
fetching 58th URL out of 253
0 files saved to: /Users/samuelpun_old/Desktop/MLfolders/ETWdata/
0 records found. 0 records updated. Total record: 6391
fetching 59th URL out of 253
289 files saved to: /Users/samuelpun_old/Desktop/MLfolders/ETWdata/
289 records fo

In [ ]:
#saveData(filePath, dbName, jsonFileName, restMasterList)

In [ ]:
#f = loadMasterJSON(filePath, jsonFileName)